In [1]:
# import of necessary libraries
import yfinance as yf
import pandas as pd

In [2]:
# connecting to the API for the companies
msft = yf.Ticker("MSFT")
aapl = yf.Ticker("AAPL")
ibm = yf.Ticker("IBM")

In [3]:
# retrieving the required company data
msft_info = msft.info
aapl_info = aapl.info
ibm_info = ibm.info

msft_pl = msft.financials
aapl_pl = aapl.financials
ibm_pl = ibm.financials

msft_bs = msft.balance_sheet
aapl_bs = aapl.balance_sheet
ibm_bs = ibm.balance_sheet

msft_cf = msft.cashflow
aapl_cf = aapl.cashflow
ibm_cf = ibm.cashflow

In [4]:
# organizing the values in a dictionary
comp_dict = [
    {"comp": "Microsoft", "info": msft_info, "pl": msft_pl, "bs": msft_bs, "cf": msft_cf},
    {"comp": "Apple", "info": aapl_info, "pl": aapl_pl, "bs": aapl_bs, "cf": aapl_cf},
    {"comp": "IBM", "info": ibm_info, "pl": ibm_pl, "bs": ibm_bs, "cf": ibm_cf}
]

In [5]:
# function to calculate the different financial ratios
def ratio_collection(info, pl, bs, cf):

    net_profit_margin = pl.loc["Net Income"][0] / pl.loc["Total Revenue"][0]
    asset_turnover = pl.loc["Total Revenue"][0] / bs.loc["Total Assets"][0]
    equity_mulitplier = bs.loc["Total Assets"][0] / bs.loc["Total Stockholder Equity"][0]
    leverage = bs.loc["Total Liab"][0] / bs.loc["Total Stockholder Equity"][0]
    buyback_ratio = cf.loc["Repurchase Of Stock"][0] * -1 / info["marketCap"]
    ROE = net_profit_margin * asset_turnover * equity_mulitplier
    EPS = pl.loc["Net Income"][0] / info["sharesOutstanding"]

    return net_profit_margin, asset_turnover, equity_mulitplier, leverage, buyback_ratio, ROE, EPS

In [6]:
# Loop that iterates through the dictionary and uses the function above
# to calculate the respective ratios.
for entity in comp_dict:
    result = list(ratio_collection(info=entity["info"], pl=entity["pl"], 
    bs=entity["bs"], cf=entity["cf"]))

    entity["net_profit_margin"] = result[0]
    entity["asset_turnover"] = result[1]
    entity["equity_mulitplier"] = result[2]
    entity["leverage"] = result[3]
    entity["buyback_ratio"] = result[4]
    entity["ROE"] = result[5]
    entity["EPS"] = result[6]

In [7]:
# organizing the results in a dataframe
df = pd.DataFrame(comp_dict)
df = df.drop(columns=["info", "pl", "bs", "cf"])
df.head()

,comp,net_profit_margin,asset_turnover,equity_mulitplier,leverage,buyback_ratio,ROE,EPS
0,Microsoft,0.366863,0.543444,2.190679,1.190679,0.018183,0.436755,9.757635
1,Apple,0.253096,1.117852,6.961537,5.961537,0.039732,1.969589,6.273722
2,IBM,0.100138,0.434474,6.983810,5.978784,0.002411,0.303846,6.407310
